In [1]:
import pandas as pd
import re
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [3]:
def clean_string(string):
    return re.sub('\'','',string)

In [4]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [6]:
df.head(2)

,Тема обращения,Суть обращения,Чиновник,Должность,Дата подачи заявления
0,Сообщение о проблеме «Неубранная городская тер...,После проведения работ на кабельной канализаци...,Ларин А.С.,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57
1,Сообщение о проблеме «Неубранная городская тер...,Очистите опору освещения. Приведите в надлежащ...,Кучма А.А.,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54


In [8]:
def search_theme(string):
    res = re.findall('«.*»',string)
    return res[0] if len(res)>0 else ""

In [ ]:
def search_address(string):
    res = re.findall('по адресу\s?(.*$)')
    return res[0] if len(res)>0 else ""

In [9]:
df["theme"] = df['Тема обращения'].apply(search_theme)

In [10]:
df.head()

,Тема обращения,Суть обращения,Чиновник,Должность,Дата подачи заявления,theme
0,Сообщение о проблеме «Неубранная городская тер...,После проведения работ на кабельной канализаци...,Ларин А.С.,начальник Объединения административно-техниче...,01 Марта 2020 в 16:57,«Неубранная городская территория»
1,Сообщение о проблеме «Неубранная городская тер...,Очистите опору освещения. Приведите в надлежащ...,Кучма А.А.,глава управы района Свиблово города Москвы,16 Января 2020 в 13:54,«Неубранная городская территория»
2,Сообщение о проблеме «Захламление территории» ...,Более двух недель лежит куча грунта в перемешк...,Сержантов С.А.,и.о. главы управы района Свиблово города Москвы,24 Августа 2019 в 15:39,«Захламление территории»
3,Сообщение о проблеме «Захламление территории» ...,"Осколки бордюрного камня спрятаны за дерево, в...",Киселев В.В.,глава управы района Свиблово города Москвы,05 Мая 2019 в 09:11,«Захламление территории»
4,Сообщение о проблеме «Несоблюдение требований ...,Прошу рассмотреть законность размещения вывеск...,Киселев В.В.,глава управы района Свиблово города Москвы,06 Февраля 2019 в 10:29,«Несоблюдение требований к размещению информац...
